In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
# load array
Data = load('Data.npy')
# print the array

In [3]:
Data=np.nan_to_num(Data)

In [4]:
Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]

for idx in range(len(Data)//39):
#     print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
    if(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any()):
        print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
        continue
    if idx< 3735:
        Xtrain_data.append(Data[39*idx:39*(idx)+38][:,:].tolist())
        Ytrain_data.append(Data[39*idx:39*(idx)+38][:,5].tolist())
    else:
        Xtest_data.append(Data[39*idx:39*(idx)+38][:,:].tolist())
        Ytest_data.append(Data[39*idx:39*(idx)+38][:,5].tolist())

Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)

Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [5]:
Xtrain_data.size()

torch.Size([141930, 6])

In [6]:
Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [7]:
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

from torch.utils.data import DataLoader, random_split
Xtrain_data,Xtest_data=random_split(Xdata, [157700-15770, 15770])
Ytrain_data,Ytest_data=random_split(Ydata, [157700-15770, 15770])

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)

In [15]:
bptt = 38
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [ ]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0

    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
            continue
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE)
        tgt=batchify(targets,BATCH_SIZE)

        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
    return losses / Xtrain_data.size(0)

In [ ]:
def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE)
            tgt=batchify(targets,BATCH_SIZE)

            logits = model(src)


            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break

            losses += loss.item()

            _,predicted=torch.max(logits,-1)
            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0/tot0*100,correct0,tot0)     
    print("Acc1:",correct1/tot1*100,correct1,tot1)     
    print("Acc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 1900
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = lstm
PATH='best_model_seq'
torch.save(best_model.state_dict(), PATH)